In [1]:
from scripts.utils import utils
from scripts.model import OR_model

import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt
import time
import gurobipy as gp
from gurobipy import GRB

In [2]:
def readData():
    print("\n\n", "-"*10, "Reading data", "-"*10)
    df = pd.read_csv('dataset.csv')

    print("Data size:", df.shape)

    print("\n\n", "-"*10, "Do precalculations", "-"*10)
    start = time.time()

    utils_ = utils(df)

    print(f"Finish, uses {time.time() - start:.5f} seconds")

    return df, utils_

In [3]:
def getAllowedMoves(df, utils):
    B = df['Block'].unique()
    P_set = set()

    for b1, b2 in itertools.product(B, repeat=2):
        o1 = utils.getRoom(b1)
        o2 = utils.getRoom(b2)
        if abs(o1 - o2) <= 10:
            P_set.add((b1,b2))

    return list(P_set)

In [4]:
df, utils = readData()
P = getAllowedMoves(df, utils)

# Hyperparameters
F = ['1']
l_v = [0,0,0,0,0]
u_v = [150,150,150,150,150]



 ---------- Reading data ----------
Data size: (1456, 7)


 ---------- Do precalculations ----------
Finish, uses 51.29623 seconds


In [5]:
model = OR_model(df=df, P=P, F=F, l_v=l_v, u_v=u_v, utils=utils)
model.init()
model.optimize()



 ---------- Creating model ----------
Set parameter Username
Set parameter LicenseID to value 2631632
Academic license - for non-commercial use only - expires 2026-03-05
New model created successfully.


 ---------- Reading sets from df and precompute ----------
Finish, uses 0.00123 seconds


 ---------- Setting decision variables ----------
Finish, uses 0.14299 seconds


 ---------- Setting constraints ----------
Setting additional constraints
Finish, uses 2.12111 seconds
Setting constraints (2)
Finish, uses 2.12813 seconds
Setting constraints (3)
Finish, uses 46.48645 seconds
Setting constraints (4)
Finish, uses 65.42948 seconds
Setting constraints (5)
Finish, uses 139.81353 seconds
---------- Setting objection ----------
Finish, uses 0.00100 seconds


 ---------- Optimizing ----------
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 5600X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 lo

In [6]:
model.saveModel("./saves/baseline")



 ---------- Saving model ----------
Model saved successfully as './saves/baseline.mps'
